# Unsupervised Machine Learning für die Nanoindentierung
Übung zur **Vorlesung Nanoindentierung**

Universität Kassel, 2026

Skript und Daten von T. Arold, modfiziert durch B. Merle

# Bereitstellen der Pakete und Funktionen
Führen Sie den unten stehenden Code mit **strg + Enter** aus um alle notwendigen Zusatzpakete und Funktionen verfügbar zu machen.

In [ ]:
# imported packages
import pandas as pd # used to handle csv- or excel files as a dataframe (table object)
import numpy as np # used for basic mathematical operations
import matplotlib.pyplot as plt # package for basic data plotting
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
import seaborn as sns # additional package with more plotting options based on pyplot


from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
import os 
os.environ["OMP_NUM_THREADS"] = "1"



# from here code for more complexe plotting functions
def _resolve_column(df: pd.DataFrame, col):
    """Resolve a column specifier for flat or MultiIndex columns.
    - col can be None, a tuple (MultiIndex), or a string.
    - Exact matches are preferred. If not found and df has MultiIndex,
      try to find columns where any level equals the string.
    """
   
    if col is None:
        return None, None  # (series, display_name)
    # tuple (explicit MultiIndex)
    if isinstance(col, tuple):
        if col in df.columns:
            return df[col], " - ".join(map(str, col))
        raise KeyError(f"Column tuple {col} not found in DataFrame columns.")
    # string case
    # direct exact match (flat columns)
    if col in df.columns:
        return df[col], str(col)
    # if MultiIndex, try exact match on any level
    if isinstance(df.columns, pd.MultiIndex):
        # exact level match
        matches = [c for c in df.columns if any(str(level) == col for level in c)]
        if len(matches) == 1:
            return df[matches[0]], " - ".join(map(str, matches[0]))
        if len(matches) > 1:
            # prefer a match where top-level equals col
            top_matches = [c for c in matches if str(c[0]) == col]
            chosen = top_matches[0] if top_matches else matches[0]
            print(f"Multiple columns match '{col}', using {chosen}.")
            return df[chosen], " - ".join(map(str, chosen))
        # try substring match (e.g., user passes 'PC 1' and column is ('PCA','PC 1'))
        substr_matches = [c for c in df.columns if any(col in str(level) for level in c)]
        if len(substr_matches) >= 1:
            chosen = substr_matches[0]
            print(f"No exact match for '{col}', using first substring match {chosen}.")
            return df[chosen], " - ".join(map(str, chosen))
    # fallback: try substring in flat columns
    flat_sub = [c for c in df.columns if col in str(c)]
    if len(flat_sub) >= 1:
        chosen = flat_sub[0]
        print(f"No exact match for '{col}', using first flat substring match {chosen}.")
        return df[chosen], str(chosen)
    raise KeyError(f"Column '{col}' not found in DataFrame columns.")

def plot_dataframe(df: pd.DataFrame,
                   x,
                   y,
                   z=None,
                   hue=None,
                   figsize=(9, 6),
                   palette=None,
                   point_size=40,
                   alpha=0.9,
                   cmap="viridis",
                   title=None):
    """
    Flexible plotting for DataFrame: automatic 2D/3D scatter depending on z.
    x, y, z, hue can be strings or tuples (for MultiIndex). If z is None -> 2D.
    Returns (fig, ax).
    """
    # Resolve columns and display names
    X, x_label = _resolve_column(df, x)
    Y, y_label = _resolve_column(df, y)
    Z, z_label = _resolve_column(df, z) if z is not None else (None, None)
    H, h_label = _resolve_column(df, hue) if hue is not None else (None, None)

    if X is None or Y is None:
        raise ValueError("x and y must be provided and resolvable to DataFrame columns.")

    x_vals = X.values
    y_vals = Y.values
    z_vals = Z.values if Z is not None else None
    hue_vals = H.values if H is not None else None

    is_3d = z_vals is not None

    fig = plt.figure(figsize=figsize)
    if is_3d:
        ax = fig.add_subplot(111, projection='3d')
    else:
        ax = fig.add_subplot(111)

    # No hue: single color
    if hue_vals is None:
        color = sns.color_palette()[0]
        if is_3d:
            sc = ax.scatter(x_vals, y_vals, z_vals, s=point_size, alpha=alpha, color=color)
        else:
            sc = ax.scatter(x_vals, y_vals, s=point_size, alpha=alpha, color=color)
    else:
        # Determine whether hue is numeric or categorical.
        # If dtype is object or string-like -> categorical.
        if pd.api.types.is_numeric_dtype(H):
            # continuous hue
            if is_3d:
                # 3D scatter with continuous color: map to RGBA
                norm = plt.Normalize(np.nanmin(hue_vals), np.nanmax(hue_vals))
                cmap_obj = plt.get_cmap(cmap)
                colors = cmap_obj(norm(hue_vals))
                sc = ax.scatter(x_vals, y_vals, z_vals, c=colors, s=point_size, alpha=alpha)
                # create a ScalarMappable for colorbar
                mappable = plt.cm.ScalarMappable(norm=norm, cmap=cmap_obj)
                mappable.set_array(hue_vals)
                cbar = fig.colorbar(mappable, ax=ax, pad=0.1)
                cbar.set_label(h_label or str(hue))
            else:
                sc = ax.scatter(x_vals, y_vals, c=hue_vals, cmap=cmap, s=point_size, alpha=alpha)
                cbar = fig.colorbar(sc, ax=ax, pad=0.1)
                cbar.set_label(h_label or str(hue))
        else:
            # categorical hue (strings or objects)
            categories, uniques = pd.factorize(hue_vals)
            n_cats = len(uniques)
            if palette is None:
                palette = sns.color_palette(n_colors=n_cats)
            colors = [palette[i % len(palette)] for i in categories]
            if is_3d:
                sc = ax.scatter(x_vals, y_vals, z_vals, c=colors, s=point_size, alpha=alpha)
            else:
                sc = ax.scatter(x_vals, y_vals, c=colors, s=point_size, alpha=alpha)
            # legend
            handles = []
            for i, lab in enumerate(uniques):
                handles.append(plt.Line2D([], [], marker='o', color=palette[i % len(palette)], linestyle='', markersize=6))
            ax.legend(handles, uniques, title=(h_label or str(hue)), bbox_to_anchor=(1.05, 1), loc='upper left')

    # Labels and title
    ax.set_xlabel(x_label or str(x))
    ax.set_ylabel(y_label or str(y))
    if is_3d:
        ax.set_zlabel(z_label or str(z))
    if title:
        ax.set_title(title)

    plt.tight_layout()
    plt.show()
    return fig, ax


# Importieren der Daten
Importieren Sie mit dem unten stehenden Codeabschnitt _(Anpassungen des Codes sind nicht notwendig)_ die Daten aus dem CSV-Ordner in ein Pandas DataFrame. Nach der Ausführung wird das DataFrame als Tabelle unterhalb des Codeblocks angezeigt. Die Daten bestehen aus 14 Spalten. 

In [ ]:
#=== dont change paramter from here ===#
# creating data frame from nanoindendation data in csv folder
df = pd.read_csv("Data/CSV/OriginalData.csv", # file path
                header = [0,1], # number of rows for column heads
                sep = ";", # separator between columns
                decimal = "," # decimal comma or decimal point
                )

# Set Indent column as Index
df.index = df[("Unnamed: 0_level_0","Unnamed: 0_level_1")]
df = df.drop(("Unnamed: 0_level_0","Unnamed: 0_level_1"), axis=1)

#visualization of the data structure

display(df)


# Plotten der mechanischen Daten
Mit der Ausführung des untenstehenden Codeblocks werden die mechanisch relevanten Daten als Mapping dargestellt. 
- Mit den `marker_options` und `colormap` können Sie die Darstellung des Plots beeinflussen. 
- Mit `indent_position = "real" oder "ideal"` legen Sie fest, ob die mechanischen Daten bezüglich ihrer Koordinaten aus der Rasterkraftmikroskopie oder der Daten aus der Nanoindentierung geplottet werden sollen.
- Mit `data_left_plot` und `data_right_plot` können Sie festlegen welche Spalten des DataFrames geplottet werden sollen

**Frage:**
Können Sie beim Vergleich beider Plots erkennen welche Indents eine Gruppe bilden könnten?

In [ ]:
# === change parameter from here === #

#choosing between ideal indent position and real indent positions
indent_position = "ideal" # <-- "real" or "ideal"

#marker options
marker_transparency = 0.75 # setting the transparency of the markers

#change the colormap of the plots
colormap = "crest_r" #<-- "viridis", "cividis", "inferno" ,"magma", "plasma", "rocket", "flare", "crest", "copper"

#selecting columns for plotting
data_left_plot = ("HARDNESS GPa","mean")
data_right_plot = ("MODULUS GPa","mean")
# ========================= #

#=== dont change paramter from here ===#

#image size
image_width_cm = 15 #change value to alter the image size
image_height_cm = 15 # change value to alter the image size

#Atomic Force Microscopy Mapping of Indentationmapping
background_image = plt.imread("Data/Images/BackGround.png")
dx,dy = -2.5,-3.2 #parameter for adjusting the background image
range_um = 50 # parameter for adjusting the background image

# automatical column selection for indent position
if indent_position == "real":
    marker_shape = ">"
    marker_size = 40 # setting the size of the markers
    x = ("x","real") # defining x axis
    y = ("y","real") # defining y axis
elif indent_position =="ideal":
    marker_shape = "s"
    marker_size = 85 # setting the size of the markers
    x = ("x","absolut") # defining x axis
    y = ("y","absolut") # defining y axis

#creating a figure object
fig, ax = plt.subplots(nrows = 1,
                       ncols = 2, #3 images next to each other
                       sharey=True)


fig.set_dpi(600) # increasing the resolution of the plot
fig.set_size_inches(image_width_cm/2.54,image_height_cm/2.54) #calcuating image size

# hardness plot 
sns.scatterplot(data = df,
                x = x,
                y = y,
                hue = data_left_plot,
                ax = ax[0], #left plot
                marker = marker_shape, #square marker
                palette = sns.color_palette(colormap,as_cmap = True), # defining the color palette
                s = marker_size, # marker size
                edgecolor = None, # remove the outline of the markers
                alpha = marker_transparency , # transparency of the markers infill
               )

if indent_position == "real":
    ax[0].imshow(background_image,
                 extent=[dx,range_um+dx,dy,range_um+dy],
                 aspect='equal',
                 zorder=-1,
                 origin = "upper"
                )

#modulus plot
sns.scatterplot(data = df,
                x = x,
                y = y,
                hue = data_right_plot,
                ax = ax[1], #middle plot
                marker = marker_shape, #square marker
                palette = sns.color_palette(colormap,as_cmap = True), # defining the color palette
                s = marker_size, # marker size
                edgecolor = None, # remove the outline of the markers
                alpha = marker_transparency , # transparency of the markers infill
                )

if indent_position == "real":
    ax[1].imshow(background_image,
                 extent=[dx,range_um+dx,dy,range_um+dy],
                 aspect='equal',
                 zorder=-1,
                 origin = "upper"
                )

for item in ax:
    item.set_aspect("equal") # ensure equidistance on x and y axis
    item.set_xlabel("X / µm")
    item.set_ylabel("Y / µm")
    sns.move_legend(loc = "lower center", bbox_to_anchor = (0.5,1), obj = item) # move legend above the corresponding plots


# Clustering nach KMeans

## Ausgangslage

Im vorherigen Schritt haben Sie die Härte- und E-Modul-Maps visuell betrachtet. Dabei lassen sich bereits Bereiche mit unterschiedlichen mechanischen Eigenschaften möglicherweise erahnen — doch eine objektive, reproduzierbare Zuordnung einzelner Indents zu Werkstoffphasen ist allein per Augenmaß kaum möglich. Genau hier setzt **Clustering** an: ein Verfahren, das Datenpunkte automatisch in Gruppen einteilt, ohne dass die Gruppenzugehörigkeit vorher bekannt sein muss.

## Was ist KMeans?

KMeans ist einer der bekanntesten Clustering-Algorithmen und gehört zum sogenannten **unüberwachten Lernen** (*unsupervised learning*). „Unüberwacht" bedeutet, dass dem Algorithmus keine gelabelten Trainingsdaten (z. B. „dieser Indent gehört zu Ferrit") vorgegeben werden. Stattdessen sucht der Algorithmus selbstständig nach Strukturen in den Daten.

Die Grundidee ist einfach: KMeans versucht, die Datenpunkte so in *k* Gruppen aufzuteilen, dass die **Summe der quadratischen Abstände** jedes Punktes zu seinem jeweiligen Clusterzentrum minimal wird. Das Clusterzentrum (auch *Zentroid* genannt) ist dabei der Mittelwert aller Punkte innerhalb eines Clusters.

## Ablauf des Algorithmus

1. **Initialisierung:** Es werden *k* zufällige Startpositionen für die Clusterzentren gewählt.
2. **Zuweisungsschritt:** Jeder Datenpunkt wird dem Clusterzentrum zugeordnet, das ihm am nächsten liegt (euklidischer Abstand).
3. **Aktualisierungsschritt:** Die Clusterzentren werden als Mittelwert aller zugeordneten Punkte neu berechnet.
4. **Iteration:** Die Schritte 2 und 3 wiederholen sich, bis sich die Zuordnungen nicht mehr ändern oder ein Abbruchkriterium erreicht ist.

Da KMeans auf euklidischen Abständen basiert, erzeugt der Algorithmus tendenziell **kugelförmige Cluster gleicher Größe**. Das ist eine wichtige Einschränkung, auf die wir später beim Vergleich mit dem Gaussian Mixture Model zurückkommen.

## Parameter

- `n_clusters`: Die Anzahl der Cluster *k*. Dieser Wert muss **vor** der Ausführung festgelegt werden. Wie man eine geeignete Anzahl bestimmt, wird in einem späteren Abschnitt behandelt.
- `feature_set`: Welche Spalten des DataFrames als Merkmale (Features) für die Gruppierung verwendet werden.

## Hinweis zur Auswahl des Feature-Sets

Die Qualität des Clusterings hängt stark davon ab, welche Merkmale übergeben werden. Dabei gibt es zwei typische Fehlerquellen:

- **Korrelierte Merkmale:** Härte in HV und Härte in GPa tragen dieselbe physikalische Information. Werden beide übergeben, wird diese Information doppelt gewichtet und verzerrt die Abstandsberechnung.
- **Nicht-physikalische Merkmale:** Die x- und y-Koordinaten der Indents beschreiben die räumliche Position auf der Probe, nicht aber die Materialeigenschaften. Werden sie dem Algorithmus übergeben, bildet er Cluster teilweise nach Ort statt nach Phase.

Es ist daher ratsam, dem KMeans-Algorithmus nur für die Fragestellung „Welche Phasen liegen vor?" relevante Merkmale zu übergeben. Nach der Ausführung des Codes wird dem DataFrame eine neue Spalte `("KMeans", "Label")` hinzugefügt, die für jeden Indent die Clusterzugehörigkeit enthält.

**Nutzen Sie für die Übung zunächst `n_clusters = 4` oder höher.**


In [ ]:
# === change parameter from here ===
n_clusters = 4  # number of clusters
feature_set = df[[
                    ("HARDNESS GPa","mean"),
                    ("MODULUS GPa","mean"),
                    ("x","real"),
                    ("y","real")
                ]].copy()
#=========================#

#=== dont change paramter from here ===#


# using the feature set defined 
X_kmeans = feature_set.copy().dropna()

# KMeans calculations 
kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=42)
labels = kmeans.fit_predict(X_kmeans).astype(int)

# using strings for labelings instead of numbers
#label_names = [f"Cluster {i+1}" for i in labels]
labels = pd.DataFrame(labels, index=X_kmeans.index)

# creating MultiIndex colum name for joining labels_df with df
#labels_df.columns = pd.MultiIndex.from_tuples([('KMeans', 'Label')])
labels.columns = pd.MultiIndex.from_tuples([('KMeans', 'Label')])
# deleting KMeans grouping if it already exists
if ('KMeans', 'Label') in df.columns:
    df = df.drop(columns=('KMeans', 'Label'))

# join KMeans Label_DataFrame with the original Dataframe df
df = df.join(labels)

display(df)

# Darstellen der KMeans-Ergebnisse

Der folgende Codeblock visualisiert die Clustering-Ergebnisse auf zwei Arten:

1. **Mapping:** Jeder Indent wird an seiner Position auf der Probe dargestellt und entsprechend seiner Clusterzugehörigkeit eingefärbt. So lässt sich erkennen, ob die gefundenen Cluster räumlich sinnvollen Bereichen (z. B. Phasen) entsprechen.
2. **Violinplot:** Für jedes Cluster werden die Verteilungen von Härte, E-Modul und $S^2/P$ dargestellt. Damit lässt sich beurteilen, ob die Cluster tatsächlich unterschiedliche mechanische Eigenschaften aufweisen oder sich stark überlappen.


In [ ]:
#=== change parameter from here ===#
#marker options
marker_shape = ">" #<-- "s" for squares, ">" for triangles
marker_size = 85 # setting the size of the markers
marker_transparency = 0.75 # setting the transparency of the markers

#choosing between ideal indent position and real indent positions
indent_position = "real" # <-- "real" or "ideal"

#change the colormap of the plots
colormap = "tab10" #<-- "tab10", "tab20", "colorblind" 

#=========================#

#=== dont change paramter from here ===#
n_cluster_kmeans = df[("KMeans","Label")].nunique()+2
#image size
image_width_cm = 15 #change value to alter the image size
image_height_cm = 15 # change value to alter the image size

#Atomic Force Microscopy Mapping of Indentationmapping
background_image = plt.imread("Data/Images/BackGround.png")
dx,dy = -2,-3.2 #parameter for adjusting the background image
range_um = 50 # parameter for adjusting the background image

# automatical column selection for indent position
if indent_position == "real":
    x = ("x","real") # defining x axis
    y = ("y","real") # defining y axis
elif indent_position =="ideal":
    x = ("x","absolut") # defining x axis
    y = ("y","absolut") # defining x axis

#creating a figure object
fig, ax = plt.subplots(nrows = 1,
                       ncols = 1, 
                       sharey=False)


fig.set_dpi(600) # increasing the resolution of the plot
fig.set_size_inches(image_width_cm/2.54,image_height_cm/2.54) #calcuating image size

# Code for Mapping
sns.scatterplot(data = df,
                x = x,
                y = y,
                hue = ("KMeans","Label"),
                ax = ax,
                marker = marker_shape, #square marker
                palette = sns.color_palette(colormap,n_cluster_kmeans)[2:], # defining the color palette
                s = marker_size, # marker size
                edgecolor = None, # remove the outline of the markers
                alpha = marker_transparency , # transparency of the markers infill
               )

ax.grid(False)
if indent_position == "real":
    ax.imshow(background_image,
                 extent=[dx,range_um+dx,dy,range_um+dy],
                 aspect='equal',
                 zorder=-1,
                 origin = "upper"
                )

ax.set_aspect("equal") # ensure equidistance on x and y axis
ax.set_xlabel("X / µm")
ax.set_ylabel("Y / µm")

sns.move_legend(loc = "lower center", bbox_to_anchor = (0.5,1), obj = ax) # move legend above the corresponding plots

#Code for Catplot

cols = [
    ("HARDNESS GPa", "mean"),
    ("MODULUS GPa", "mean"),
    ("S2overP", "mean"),
    ("KMeans", "Label"),
]

df_sel = df[cols].copy()
df_sel.columns = ["Hardness", "Modulus", "S2overP", "Cluster"]

df_long = df_sel.melt(
    id_vars="Cluster",
    var_name="Property",
    value_name="Value"
)

g = sns.catplot(
        data=df_long,
        x="Cluster",
        y="Value",
        col="Property",      # drei Plots nebeneinander
        kind="violin",          # Mittelwerte als Balken
        sharey=False,
        hue = "Cluster", # unterschiedliche Skalen erlaubt
        palette = sns.color_palette(colormap,n_cluster_kmeans)[2:]
        )
g.figure.set_dpi(600)
g.figure.set_size_inches(20/2.5,7/2.54)
g._legend.remove()

for ax in g.axes.flat:
    ax.set_axisbelow(True)     
    ax.grid(True, linestyle="--", alpha=1,linewidth = 2)
    ax.tick_params(axis="x", rotation=45)
plt.show()

## Bewertung der Ergebnisse

Betrachten Sie die Violinplots der einzelnen Cluster. Es sollte auffallen, dass sich die mechanischen Eigenschaften der Gruppen **kaum voneinander unterscheiden** — die Verteilungen überlappen stark. Das deutet darauf hin, dass das Clustering in dieser Form keine sinnvolle Phasentrennung liefert.

Mögliche Ursachen sind:

- **Ungeeignetes Feature-Set:** Wurden z. B. korrelierte oder nicht-physikalische Merkmale mit übergeben?
- **Falsche Clusteranzahl:** Stimmt *k* nicht mit der tatsächlichen Phasenanzahl überein?
- **Zu hohe Dimensionalität:** Arbeitet der Algorithmus in zu vielen Dimensionen, was die Abstandsberechnung erschwert?

In den nächsten Schritten werden wir diese Probleme systematisch angehen: zuerst mit einer **Dimensionsreduktion (PCA)**, dann mit einer **datengestützten Bestimmung der Clusteranzahl**.

# Principal Component Analysis (PCA)

## Motivation

Im vorherigen Abschnitt wurde der KMeans-Algorithmus direkt auf die Rohdaten angewendet. Dabei sind zwei Probleme aufgetreten:

1. **Zu viele oder ungeeignete Merkmale:** Jede Spalte im Feature-Set entspricht einer Dimension im Datenraum. Werden z. B. fünf Spalten übergeben, arbeitet der Algorithmus in einem fünfdimensionalen Raum. Das erschwert nicht nur die Visualisierung (ab vier Dimensionen nicht mehr darstellbar), sondern kann auch die Clusterbildung verschlechtern — insbesondere wenn Merkmale stark korreliert oder für die Fragestellung irrelevant sind.

2. **Korrelierte Merkmale verzerren Abstände:** Härte in GPa und Härte in HV tragen im Grunde dieselbe physikalische Information. Werden beide Spalten übergeben, wird diese Information doppelt gewichtet, was die Abstandsberechnung im Clusteralgorithmus verzerrt.

Die **Principal Component Analysis** (PCA) löst beide Probleme gleichzeitig.

## Was macht die PCA?

Die PCA ist ein Verfahren aus der linearen Algebra, das einen hochdimensionalen Datensatz auf ein neues, orthogonales Koordinatensystem transformiert. Die neuen Achsen — die sogenannten **Hauptkomponenten** (Principal Components, PC) — werden so gewählt, dass:

- **PC1** die Richtung im Datenraum beschreibt, entlang derer die Varianz (also die Streuung) der Daten am größten ist,
- **PC2** senkrecht auf PC1 steht und die zweitgrößte Varianz erfasst,
- und so weiter für jede weitere Komponente.

Anschaulich kann man sich das wie folgt vorstellen: Stellen Sie sich eine Punktwolke in 3D vor — z. B. Messwerte aus der Nanoindentierung mit Härte, E-Modul und $S^2/P$ als Achsen. Die PCA legt zunächst eine neue Achse (PC1) durch die Richtung der größten Streuung dieser Wolke. PC2 steht senkrecht dazu und erfasst die verbliebene Streuung. Wenn die Punktwolke eher flach ist (wie eine Scheibe), reichen zwei Hauptkomponenten bereits aus, um die wesentliche Struktur der Daten zu beschreiben — die dritte Dimension kann verworfen werden, ohne relevante Information zu verlieren.

## Warum ist das nützlich?

Die PCA liefert zwei zentrale Ergebnisse:

**1. Erklärte Varianz (Explained Variance Ratio)**
Für jede Hauptkomponente wird angegeben, welcher Anteil der Gesamtstreuung durch diese Achse erklärt wird. Erklären z. B. PC1 und PC2 zusammen bereits 99 % der Varianz, dann steckt nahezu die gesamte Information des ursprünglichen Datensatzes in nur zwei Dimensionen. Die übrigen Komponenten können ohne nennenswerten Informationsverlust weggelassen werden. Das Ergebnis ist ein **dimensionsreduziertes Feature-Set**, das für den Clusteralgorithmus einfacher und robuster zu verarbeiten ist.

**2. Ladungen (Loadings)**
Die Loadings-Tabelle zeigt, wie stark jedes ursprüngliche Merkmal zu einer Hauptkomponente beiträgt. Ein hoher Absolutwert bedeutet eine starke Beteiligung, ein Wert nahe null bedeutet, dass dieses Merkmal für die jeweilige Komponente kaum relevant ist. Merkmale, die auf allen Komponenten nahe null liegen, tragen wenig zur Datenstruktur bei und können aus dem Feature-Set entfernt werden.

## Wichtig: Standardisierung

Vor der PCA werden die Daten **standardisiert** (z-Transformation): Jede Spalte wird so skaliert, dass sie einen Mittelwert von 0 und eine Standardabweichung von 1 besitzt. Das ist notwendig, weil die PCA auf Varianzen basiert. Ohne Standardisierung würde ein Merkmal mit großem Wertebereich (z. B. E-Modul in GPa) die Analyse dominieren, obwohl es nicht unbedingt informativer ist als ein Merkmal mit kleinerem Wertebereich (z. B. $S^2/P$). Die Standardisierung wird im Code automatisch mit `StandardScaler()` durchgeführt.

## Hinweise zur Anwendung

- **Nicht-physikalische Größen ausschließen:** Die x- und y-Koordinaten der Indents sind keine Materialkennwerte. Werden sie der PCA übergeben, fließt die räumliche Position in die Hauptkomponenten ein und verfälscht die Phasentrennung. Schließen Sie diese Spalten daher aus.
- Physikalisch sinnvolle Merkmale zur Phasentrennung sind z. B.: Härte (GPa), E-Modul (GPa) und die quadrierte Steifigkeit bezogen auf die Last ($S^2/P$).

**Parameter:**
- Mit `features` definieren Sie, welche Spalten an die PCA übergeben werden.
- Mit `n_components` legen Sie die Anzahl der Hauptkomponenten fest (belassen Sie den Wert zunächst auf 2).


In [ ]:
#=== change parameter ===#
#selecting important columns for clustering
features = df[[
                ("HARDNESS GPa","mean"),
                ("MODULUS GPa","mean"),
                ("S2overP","mean"),
#            	("x","real"),
#                ("y","real")
                ]].copy()

n_components = 2 #<-- determination of the dimension of the data space
#=========================#

#=== don't change parameters from here ===#

#deleting indents with not data (Indent 15, Indent 67) 
features.dropna(inplace = True)

# scale data very important for correct results!
scaler = StandardScaler() 
X_scaled = scaler.fit_transform(features)

# PCA on scaled data
pca = PCA(n_components=n_components) 
X_pca = pca.fit_transform(X_scaled)

fig = plt.figure(figsize=(8, 6)) 
if X_pca.shape[1] == 3:
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2], c='steelblue', s=40)
    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')
    ax.set_zlabel('PC3')
    ax.set_title('PCA Scatterplot (3D)')
    plt.show()
    print("Erklärte Varianzanteile:", pca.explained_variance_ratio_,"\n")
    loadings = pd.DataFrame(pca.components_.T, columns=['PCA1', 'PCA2',"PCA3"], index=features.columns ) 
    print(loadings)

else:
    plt.scatter(X_pca[:, 0], X_pca[:, 1], c='steelblue', s=40)
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.title('PCA Scatterplot (2D)')
    plt.show()
    print("Erklärte Varianzanteile:", pca.explained_variance_ratio_, "\n")
    loadings = pd.DataFrame(pca.components_.T, columns=['PC 1', 'PC 2'], index=features.columns ) 
    print(loadings)

# deleting PCA columns in original data dataframe if present 
if 'PCA' in df.columns.get_level_values(0):
    df = df.drop(columns='PCA', level=0)

# make dataframe for PCA1 and PCA2 and maybe PCA3 axis
pca_columns = [f'PC{i+1}' for i in range(X_pca.shape[1])]
pca_df = pd.DataFrame(X_pca, columns=pca_columns, index=features.index)

# transform to multi index column ("PCA","PC 1),("PC","PC 2"),...
pca_df.columns = pd.MultiIndex.from_product([['PCA'], pca_columns])

# join pca_df with orignal DataFrame df by Index "Indent Nr"
df = df.join(pca_df)

## Interpretation der PCA-Ergebnisse

**Erklärte Varianzanteile:** Die ausgegebenen Werte zeigen, welcher Anteil der Gesamtvarianz durch jede Hauptkomponente erklärt wird. Beschreiben PC1 und PC2 zusammen bereits ca. 99 % der Varianz, ist die Reduktion auf zwei Dimensionen gerechtfertigt — eine Erhöhung von `n_components` auf 3 würde keinen nennenswerten Informationsgewinn bringen und die Dimensionalität unnötig erhöhen.

**Loadings-Tabelle:** Die Tabelle zeigt die Gewichtung jedes Merkmals auf den Hauptkomponenten. Beispielhaft:

- Auf **PC1** besitzen Härte und E-Modul hohe Absolutwerte → PC1 trennt vor allem nach diesen mechanischen Eigenschaften.
- Auf **PC2** besitzen E-Modul und $S^2/P$ hohe Absolutwerte → PC2 erfasst eine zusätzliche Variation, die durch PC1 allein nicht abgebildet wird.

Merkmale mit Werten nahe null auf allen Komponenten tragen wenig zur Datenstruktur bei und können aus dem Feature-Set entfernt werden.

**Hinweis:**
Sollten die Ergebnisse nicht zufriedenstellend sein (z. B. unnötige Komponenten oder irrelevante Merkmale im Feature-Set), können Sie `features` und `n_components` anpassen und den Codeblock erneut ausführen.


# Bestimmen einer geeigneten Clusteranzahl

Unüberwachte Clustering-Algorithmen wie KMeans oder das Gaussian Mixture Model benötigen die Angabe, in wie viele Cluster die Daten aufgeteilt werden sollen. In der Praxis ist diese Anzahl jedoch oft nicht bekannt — insbesondere wenn man die Daten explorativ nach Mustern durchsucht, etwa um unbekannte Phasen in einer Werkstoffprobe zu identifizieren.

Es gibt daher quantitative Kriterien, die bei der Wahl einer geeigneten Clusteranzahl helfen. Der folgende Codeblock führt den KMeans-Algorithmus wiederholt für Clusteranzahlen von *k* = 2 bis *k* = 10 aus und wertet zwei solche Kriterien aus: den **Silhouette Score** und die **Inertia** (Elbow-Plot).

## Silhouette Score

Der Silhouette Score bewertet für jeden einzelnen Datenpunkt, wie gut er zu seinem eigenen Cluster passt im Vergleich zum nächstgelegenen fremden Cluster. Formal berechnet er sich aus zwei Größen:

- **a**: der mittlere Abstand des Punktes zu allen anderen Punkten im selben Cluster (Maß für die Kompaktheit).
- **b**: der mittlere Abstand des Punktes zu allen Punkten im nächstgelegenen fremden Cluster (Maß für die Trennung).

Daraus ergibt sich:  $s = \frac{b - a}{\max(a, b)}$

Der Wert liegt zwischen −1 und +1:

- **Nahe +1:** Der Punkt liegt klar innerhalb seines Clusters und weit von anderen Clustern entfernt → gute Trennung.
- **Nahe 0:** Der Punkt liegt an der Grenze zwischen zwei Clustern → uneindeutige Zuordnung.
- **Negativ:** Der Punkt ist wahrscheinlich dem falschen Cluster zugeordnet.

Der über alle Punkte gemittelte Silhouette Score dient als Gesamtmaß für die Clusterqualität. Die Clusteranzahl mit dem **höchsten** Silhouette Score ist in der Regel eine gute Wahl.

## Inertia und Elbow-Plot

Die **Inertia** (auch *Within-Cluster Sum of Squares*, WCSS) ist die Summe der quadratischen Abstände aller Punkte zu ihrem jeweiligen Clusterzentrum:

$$\text{Inertia} = \sum_{i=1}^{k} \sum_{x \in C_i} \|x - \mu_i\|^2$$

Niedrige Werte bedeuten kompakte Cluster. Allerdings sinkt die Inertia **zwangsläufig** mit steigender Clusteranzahl — im Extremfall (ein Cluster pro Punkt) ist sie null. Deshalb kann die Inertia nicht als absolutes Kriterium dienen.

Stattdessen wird sie im **Elbow-Plot** gegen die Clusteranzahl aufgetragen. Gesucht wird ein „Knick" (Ellbogen) in der Kurve: Ab diesem Punkt bringt ein zusätzliches Cluster nur noch eine geringe Reduktion der Inertia — der Informationsgewinn rechtfertigt die höhere Komplexität nicht mehr. In der Praxis ist der Knick allerdings nicht immer eindeutig erkennbar, weshalb der Elbow-Plot am besten in Kombination mit dem Silhouette Score interpretiert wird.

**Hinweis:** Mit `feature_set` legen Sie fest, ob das dimensionsreduzierte PCA-Feature-Set oder das originale Feature-Set verwendet wird. Da bereits eine PCA durchgeführt wurde, empfiehlt es sich, `feature_set = "PCA"` zu wählen.


In [ ]:
#=== Change from here ===#
feature_set = "PCA" #<-- "PCA" or "features"
#========================#

#=== Dont change from here ===#
max_number_cluster = 10
silhouette_scores = []
inertias = []
ks = []

# Select data basis
if feature_set == 'PCA':
    if ("PCA","PC1") not in df.columns:
        raise ValueError(f"Für 'feature_set = PCA' bitte zuvor die PCA-Analyse (Code:Durchführen PCA) ausführen!")
    X = df['PCA'].dropna()
elif feature_set == 'features':
    X = features.copy()
else:
    raise ValueError("features_source must be 'PCA' or 'features'.")

for k in range(2,max_number_cluster):
    kmeans = KMeans(n_clusters=k, n_init='auto', random_state=42)
    labels = kmeans.fit_predict(X)
    inertia = kmeans.inertia_
    inertias.append(inertia)
    ks.append(k)

# Silhouette Score is only defined for k > 1
    score = silhouette_score(X, labels)
    silhouette_scores.append(score)
    print(f"k = {k}, Inertia = {inertia:.2f}, Silhouette Score = {score:.3f}")

# Plotting
fig, ax1 = plt.subplots(figsize=(9, 5))

color1 = 'tab:blue'
ax1.set_xlabel('Number of Clusters (k)')
ax1.set_ylabel('Inertia (Elbow)', color=color1)
ax1.plot(ks, inertias, marker='o', color=color1, label='Inertia')
ax1.tick_params(axis='y', labelcolor=color1)

ax2 = ax1.twinx()  # second y-axis for Silhouette Score
color2 = 'tab:green'
ax2.set_ylabel('Silhouette Score', color=color2)
ax2.plot(ks, silhouette_scores, marker='s', linestyle='--', color=color2, label='Silhouette Score')
ax2.tick_params(axis='y', labelcolor=color2)

plt.title('KMeans: Elbow Plot & Silhouette Score')
fig.tight_layout()
plt.grid(True)
plt.show()


In dem obigen Plot sind der Elbow-Plot und der Silhouette Score überlagert in Abhängigkeit von der Cluster-Anzahl dargestellt.

Beurteilen Sie anhand des Plots, welche Clustergröße für den KMeans-Algorithmus am besten zu verwenden ist.

# Optimiertes Clustering nach KMeans

Im ersten KMeans-Durchlauf (oben) wurden die Rohdaten direkt verwendet, was zu schlecht getrennten Clustern führte. Inzwischen wurden zwei Verbesserungen vorgenommen:

1. **Dimensionsreduktion durch PCA:** Das Feature-Set wurde auf die Hauptkomponenten PC1 und PC2 reduziert, die den Großteil der Varianz abbilden.
2. **Datengestützte Wahl von *k*:** Silhouette Score und Elbow-Plot liefern eine fundierte Grundlage für die Clusteranzahl.

Der folgende Codeblock führt das KMeans-Clustering auf dem PCA-Feature-Set durch. Passen Sie `n_clusters` anhand Ihrer Auswertung des Silhouette Scores und des Elbow-Plots an.


In [ ]:
# === change parameter from here ===#
# KMeans Clustersize
n_clusters = 4  # number of clusters

# Plot Options
#marker options
marker_shape = ">" #<-- "s" for squares, ">" for triangles
marker_size = 85 # setting the size of the markers
marker_transparency = 0.75 # setting the transparency of the markers

#choosing between ideal indent position and real indent positions
indent_position = "real" # <-- "real" or "ideal"

#change the colormap of the plots
colormap = "tab10" #<-- "tab10", "tab20", "colorblind" 
#=========================#

#=== dont change paramter from here ===#

#Code for KMeans
feature_pca = df.xs("PCA",axis=1,level=0).copy()
# using the feature set defined 
X_kmeans = feature_pca.dropna()

# KMeans calculations 
kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=42)
labels = kmeans.fit_predict(X_kmeans).astype(int)

# using strings for labelings instead of numbers
#label_names = [f"Cluster {i+1}" for i in labels]
labels = pd.DataFrame(labels, index=X_kmeans.index)

# creating MultiIndex colum name for joining labels_df with df
#labels_df.columns = pd.MultiIndex.from_tuples([('KMeans', 'Label')])
labels.columns = pd.MultiIndex.from_tuples([('KMeans', 'Label')])
# deleting KMeans grouping if it already exists
if ('KMeans', 'Label') in df.columns:
    df = df.drop(columns=('KMeans', 'Label'))

# join KMeans Label_DataFrame with the original Dataframe df
df = df.join(labels)

#Code for plotting

n_cluster_kmeans = df[("KMeans","Label")].nunique()+2

#image size
image_width_cm = 15 #change value to alter the image size
image_height_cm = 15 # change value to alter the image size

#Atomic Force Microscopy Mapping of Indentationmapping
background_image = plt.imread("Data/Images/BackGround.png")
dx,dy = -2,-3.2 #parameter for adjusting the background image
range_um = 50 # parameter for adjusting the background image

# automatical column selection for indent position
if indent_position == "real":
    x = ("x","real") # defining x axis
    y = ("y","real") # defining y axis
elif indent_position =="ideal":
    x = ("x","absolut") # defining x axis
    y = ("y","absolut") # defining y axis

#creating a figure object
fig_map_kmeans, ax = plt.subplots(nrows = 1,
                       ncols = 1, 
                       sharey=False)


fig_map_kmeans.set_dpi(600) # increasing the resolution of the plot
fig_map_kmeans.set_size_inches(image_width_cm/2.54,image_height_cm/2.54) # calculating image size

# Code for Mapping
map_kmeans = sns.scatterplot(data = df,
                x = x,
                y = y,
                hue = ("KMeans","Label"),
                ax = ax,
                marker = marker_shape, #square marker
                palette = sns.color_palette(colormap,n_cluster_kmeans)[2:], # defining the color palette
                s = marker_size, # marker size
                edgecolor = None, # remove the outline of the markers
                alpha = marker_transparency , # transparency of the markers infill
               )

ax.grid(False)
if indent_position == "real":
    ax.imshow(background_image,
                 extent=[dx,range_um+dx,dy,range_um+dy],
                 aspect='equal',
                 zorder=-1,
                 origin = "upper"
                )

ax.set_aspect("equal") # ensure equidistance on x and y axis
sns.move_legend(loc = "lower center", bbox_to_anchor = (0.5,1), obj = ax) # move legend above the corresponding plots

#Code for Catplot

cols = [
    ("HARDNESS GPa", "mean"),
    ("MODULUS GPa", "mean"),
    ("S2overP", "mean"),
    ("KMeans", "Label"),
]

df_sel = df[cols].copy()
df_sel.columns = ["Hardness", "Modulus", "S2overP", "Cluster"]

df_long = df_sel.melt(
    id_vars="Cluster",
    var_name="Property",
    value_name="Value"
)

Cat_kmeans = sns.catplot(
        data=df_long,
        x="Cluster",
        y="Value",
        col="Property",      # drei Plots nebeneinander
        kind="violin",          # Mittelwerte als Balken
        sharey=False,
        hue = "Cluster", # unterschiedliche Skalen erlaubt
        palette = sns.color_palette(colormap,n_cluster_kmeans)[2:]
        )
Cat_kmeans.figure.set_dpi(600)
Cat_kmeans.figure.set_size_inches(20/2.5,7/2.54)
Cat_kmeans._legend.remove()

for ax in Cat_kmeans.axes.flat:
    ax.set_axisbelow(True)     
    ax.grid(True, linestyle="--", alpha=1,linewidth = 2)
    ax.tick_params(axis="x", rotation=45)
plt.show()

## Vergleich: Vorher vs. Nachher

Vergleichen Sie das aktuelle Mapping und die Violinplots mit den Ergebnissen der unoptimierten KMeans-Analyse von weiter oben. Achten Sie insbesondere auf:

- **Räumliche Kohärenz:** Bilden die Cluster jetzt zusammenhängende Bereiche auf der Probe?
- **Mechanische Trennschärfe:** Überlappen die Verteilungen von Härte, E-Modul und $S^2/P$ zwischen den Clustern weniger als zuvor?

Es sollte deutlich werden, dass die Kombination aus PCA und datengestützter Clusteranzahl zu einer erheblich besseren Phasentrennung führt.

# Clustering nach Gaussian Mixture Model (GMM)

## Motivation

KMeans minimiert quadratische Abstände zu Clusterzentren und erzeugt dadurch tendenziell **kugelförmige Cluster gleicher Größe**. In der Realität sind Phasenbereiche in Werkstoffproben jedoch nicht immer gleich groß oder symmetrisch verteilt. Das **Gaussian Mixture Model** (GMM) bietet hier eine flexiblere Alternative.

## Was ist ein Gaussian Mixture Model?

GMM gehört ebenfalls zum unüberwachten Lernen, verfolgt aber einen grundlegend anderen Ansatz als KMeans. Die Grundidee: Man nimmt an, dass die beobachteten Datenpunkte aus einer **Mischung mehrerer Normalverteilungen** stammen — jede Normalverteilung repräsentiert ein Cluster.

Jede dieser Normalverteilungen wird durch drei Größen beschrieben:

- **Mittelwertvektor $\mu$:** das Zentrum des Clusters im Merkmalsraum.
- **Kovarianzmatrix $\Sigma$:** beschreibt die Form, Ausdehnung und Orientierung des Clusters. Im Gegensatz zu KMeans, das implizit von kugelförmigen Clustern ausgeht, kann GMM durch die freie Kovarianzmatrix auch **elliptische Cluster** mit beliebiger Orientierung modellieren.
- **Mischungsgewicht $\pi$:** der Anteil der Datenpunkte, der zu diesem Cluster gehört.

## Ablauf des Algorithmus (EM-Algorithmus)

GMM wird mit dem **Expectation-Maximization (EM)**-Algorithmus optimiert, einem iterativen Verfahren:

1. **Initialisierung:** Für jedes Cluster werden Startwerte für $\mu$, $\Sigma$ und $\pi$ festgelegt.
2. **E-Schritt (Expectation):** Für jeden Datenpunkt wird die **Zugehörigkeitswahrscheinlichkeit** zu jedem Cluster berechnet. Ein Punkt kann also z. B. mit 80 % zu Cluster 1 und mit 20 % zu Cluster 2 gehören — im Gegensatz zu KMeans, das immer eine harte Zuordnung vornimmt.
3. **M-Schritt (Maximization):** Die Parameter $\mu$, $\Sigma$ und $\pi$ jedes Clusters werden so aktualisiert, dass die **Gesamtwahrscheinlichkeit (Likelihood)** der Daten maximal wird.
4. **Iteration:** E- und M-Schritt wechseln sich ab, bis die Parameter konvergieren.
5. **Clusterzuweisung:** Jedem Punkt wird das Cluster mit der höchsten Zugehörigkeitswahrscheinlichkeit zugewiesen.

## Bestimmung der Clusteranzahl für GMM

Da GMM und KMeans unterschiedliche Modelle verwenden, muss die geeignete Clusteranzahl **für jeden Algorithmus separat** bestimmt werden. Der folgende Codeblock wertet drei Kriterien aus:

**Silhouette Score** — wie beim KMeans (siehe oben).

**BIC (Bayesian Information Criterion) und AIC (Akaike Information Criterion):**
BIC und AIC sind Informationskriterien, die die **Modellgüte** (wie gut das Modell die Daten erklärt) gegen die **Modellkomplexität** (Anzahl der Parameter) abwägen. Beide basieren auf der Log-Likelihood, bestrafen aber zusätzlich die Anzahl freier Parameter:

- **AIC** bestraft die Komplexität moderat und tendiert daher manchmal zu Modellen mit mehr Clustern.
- **BIC** bestraft die Komplexität stärker (abhängig von der Stichprobengröße) und bevorzugt daher sparsamere Modelle mit weniger Clustern.

In beiden Fällen gilt: **kleiner ist besser**. Gesucht wird die Clusteranzahl, bei der BIC und/oder AIC ein Minimum erreichen — idealerweise in Übereinstimmung mit einem hohen Silhouette Score.

**Hinweis:** Mit `feature_set` legen Sie fest, ob das PCA-Feature-Set oder das originale Feature-Set verwendet wird.


In [ ]:
# === Change from here === #
feature_set = "PCA" #<-- "PCA" or "features"
# ======================== #

silhouette_scores = []
neg_log_likelihoods = []
bic_scores = []
aic_scores = []
ks = []

# Select data basis
if feature_set == 'PCA':
    X = df['PCA'].dropna()
elif feature_set == 'features':
    X = features.copy()
else:
    raise ValueError("features_source must be 'PCA' or 'features'.")

# Compute metrics for different k
for k in range(2, max_number_cluster):
    gmm = GaussianMixture(n_components=k, random_state=42, n_init=5)
    gmm.fit(X)
    labels = gmm.predict(X)
    
    neg_ll = -gmm.score(X) * len(X)
    bic = gmm.bic(X)
    aic = gmm.aic(X)
    score = silhouette_score(X, labels)
    
    neg_log_likelihoods.append(neg_ll)
    bic_scores.append(bic)
    aic_scores.append(aic)
    silhouette_scores.append(score)
    ks.append(k)

# === Plot 1: -Log-Likelihood + Silhouette ===
fig, ax1 = plt.subplots(figsize=(9,5))

color1 = 'tab:purple'
ax1.set_xlabel('Number of Clusters (k)')
ax1.set_ylabel('− Log-Likelihood', color=color1)
ax1.plot(ks, neg_log_likelihoods, marker='o', color=color1, label='− Log-Likelihood')
ax1.tick_params(axis='y', labelcolor=color1)
ax1.grid(True)

ax2 = ax1.twinx()
color2 = 'tab:green'
ax2.set_ylabel('Silhouette Score', color=color2)
ax2.plot(ks, silhouette_scores, marker='s', linestyle='--', color=color2, label='Silhouette Score')
ax2.tick_params(axis='y', labelcolor=color2)

# Combined legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

plt.title('GMM: −Log-Likelihood & Silhouette Score')
fig.tight_layout()
plt.show()

# === Plot 2: BIC + AIC ===
plt.figure(figsize=(9,5))
plt.plot(ks, bic_scores, marker='x', linestyle='--', color='red', label='BIC')
plt.plot(ks, aic_scores, marker='^', linestyle='--', color='orange', label='AIC')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Information Criteria')
plt.title('GMM: BIC & AIC')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

## Interpretation

Betrachten Sie die Kriterien-Plots (Silhouette Score, BIC, AIC) und bestimmen Sie die geeignete Clusteranzahl für den GMM-Algorithmus.

## Vergleich KMeans vs Gaussian Mixture Model

| Eigenschaft | KMeans | GMM |
|---|---|---|
| Clusterform | kugelförmig, gleich groß | beliebig elliptisch |
| Optimiert | Summe quadratischer Abstände | Likelihood |
| Empfindlich auf | Ausreißer, ungleiche Clustergrößen | Überanpassung bei wenig Daten |

**Hinweis:** Überprüfen Sie vor der Ausführung des nächsten Codeblocks, ob der Parameter `n_components` der gewählten Clusteranzahl entspricht, und passen Sie den Wert gegebenenfalls an.

Die anschließend erzeugten Plots stellen die Ergebnisse von KMeans und GMM direkt gegenüber. Beurteilen Sie:

- **Clusteranzahl:** Führen beide Algorithmen zur gleichen optimalen Anzahl an Clustern?
- **Mechanische Eigenschaften:** Unterscheiden sich die ermittelten Härte-, E-Modul- und $S^2/P$-Verteilungen der Cluster zwischen den Algorithmen?
- **Räumliche Verteilung:** Stimmen die Mappings beider Algorithmen überein, oder gibt es Bereiche, in denen sie unterschiedliche Zuordnungen liefern?


In [ ]:
# === change parameter from here ===#
# GMM Cluster number
n_components = 4  # number of clusters

# Plot Options
marker_shape = ">"  # "<" for triangles, "s" for squares
marker_size = 40
marker_transparency = 0.75

# choosing between ideal indent position and real indent positions
indent_position = "real"  # "real" or "ideal"

# colormap
colormap = "tab10"  # "tab10", "tab20", "colorblind"
#=========================#

#=== don't change from here ===#

# the allowed shape of the clusters
covariance_type = "full" #<-- full, tied, diag, spherical

# --- Prepare feature set ---
feature_pca = df.xs("PCA", axis=1, level=0).copy()
X_gmm = feature_pca.dropna()

# --- Fit GMM ---
gmm = GaussianMixture(n_components=n_components, random_state=42, n_init=5,covariance_type = covariance_type)
labels = gmm.fit_predict(X_gmm).astype(int)

# convert to DataFrame and MultiIndex column for joining
labels = pd.DataFrame(labels, index=X_gmm.index)
labels.columns = pd.MultiIndex.from_tuples([('GMM', 'Label')])

# remove existing GMM column if exists
if ('GMM', 'Label') in df.columns:
    df = df.drop(columns=('GMM', 'Label'))

# join labels with original df
df = df.join(labels)

# --- Scatterplot / Mapping ---

n_clusters_kmeans = df[("KMeans", "Label")].nunique() + 2
n_clusters_gmm = df[("GMM","Label")].nunique() + 2


image_width_cm = 15
image_height_cm = 15
background_image = plt.imread("Data/Images/BackGround.png")
dx, dy = -2, -3.2
range_um = 50

if indent_position == "real":
    x = ("x","real")
    y = ("y","real")
elif indent_position == "ideal":
    x = ("x","absolut")
    y = ("y","absolut")

fig_map, ax = plt.subplots(ncols=2,
                           nrows=1,
                          sharey=True)
fig_map.set_dpi(600)
fig_map.set_size_inches(image_width_cm/2.5, image_height_cm/2.54)

sns.scatterplot(
    data=df,
    x=x,
    y=y,
    hue=("KMeans","Label"),
    ax=ax[0],
    marker=marker_shape,
    palette=sns.color_palette(colormap,n_clusters_kmeans)[2:],
    s=marker_size,
    edgecolor=None,
    alpha=marker_transparency
)



if indent_position == "real":
    ax[0].imshow(
        background_image,
        extent=[dx, range_um+dx, dy, range_um+dy],
        aspect='equal',
        zorder=-1,
        origin="upper"
    )
ax[0].set_axisbelow(True)
ax[0].set_aspect("equal")

sns.scatterplot(
    data=df,
    x=x,
    y=y,
    hue=("GMM","Label"),
    ax=ax[1],
    marker=marker_shape,
    palette=sns.color_palette(colormap,n_clusters_gmm)[2:],
    s=marker_size,
    edgecolor=None,
    alpha=marker_transparency
)



if indent_position == "real":
    ax[1].imshow(
        background_image,
        extent=[dx, range_um+dx, dy, range_um+dy],
        aspect='equal',
        zorder=-1,
        origin="upper"
    )
ax[1].set_axisbelow(True)
ax[1].set_aspect("equal")
for item in ax:
    sns.move_legend(loc="lower center", bbox_to_anchor=(0.5,1), obj=item)
plt.show()



# Spalten für Properties
properties = [("HARDNESS GPa","mean"),("MODULUS GPa","mean"), ("S2overP","mean")]
n_props = len(properties)

# Figure mit 2 Zeilen (KMeans, GMM) und n_props Spalten
fig, axes = plt.subplots(nrows=2, ncols=n_props, figsize=(4*n_props, 8), sharey=False)

# Farbpalette
palette_kmeans = sns.color_palette(colormap,n_clusters_kmeans)[2:]
palette_gmm = sns.color_palette(colormap,n_clusters_gmm)[2:]

# --- Erste Zeile: KMeans ---
for i, prop in enumerate(properties):

    y = df[prop].dropna()
    sns.violinplot(
        data=df,
        x=df[("KMeans","Label")],
        y=y,
        hue=df[("KMeans","Label")],
        ax=axes[0, i],
        palette=palette_kmeans,
        split=False
    )
    axes[0, i].set_title(f"{prop} (KMeans)")
    axes[0, i].tick_params(axis="x", rotation=45)
    axes[0, i].set_xlabel("")
    axes[0, i].grid(True, linestyle="--", alpha=0.4)
    axes[0, i].set_axisbelow(True)   
    axes[0, i].legend_.remove()

# --- Zweite Zeile: GMM ---
for i, prop in enumerate(properties):
    y = df[prop].dropna()
    sns.violinplot(
        data=df,
        x=df[("GMM","Label")],
        y=y,
        hue=df[("GMM","Label")],
        ax=axes[1, i],
        palette=palette_gmm,
        split=False
    )
    axes[1, i].set_title(f"{prop} (GMM)")
    axes[1, i].tick_params(axis="x", rotation=45)
    axes[1, i].set_xlabel("")
    axes[1, i].grid(True, linestyle="--", alpha=0.4)
    axes[1, i].set_axisbelow(True)   
    axes[1, i].legend_.remove()

# Optional: globaler Titel
fig.suptitle("Cluster Properties: KMeans vs GMM", fontsize=16, y=1)

plt.tight_layout()
plt.show()

plot_dataframe(df, #<-- keep it 
               x=('PCA','PC1'), # choose x-axis
               y=("PCA",'PC2'), # choose y-axis
               z = None, # choose z-axis or set it to None for 2D plot
               hue=('KMeans','Label') # 
              ) 

plot_dataframe(df, #<-- keep it 
               x=('PCA','PC1'), # choose x-axis
               y=("PCA",'PC2'), # choose y-axis
               z = None, # choose z-axis or set it to None for 2D plot
               hue=('GMM','Label') # 
              ) 